In [7]:
import pandas as pd                         # 데이터 분석 패키지
import numpy as np                         # 계산 패키지
import matplotlib.pyplot as plt             # 데이터 시각화 패키지

# 한글 깨짐 방지 
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

# 
from matplotlib import rc
import seaborn as sns
%matplotlib inline

#rc('font', family = 'AppleGothic')
#plt.rcParams['axes.unicode_minus'] = False


# 데이터 불러오기
cafeteria = pd.read_csv('cafeteria/전국무료급식소표준데이터.csv', encoding = 'UTF-8')
restaurant = pd.read_csv('cafeteria/전국아동복지급식정보표준데이터.csv',encoding = 'UTF-8')

In [3]:
print(cafeteria.head())
print(cafeteria.info())

           시설명                      소재지도로명주소  \
0      동산사회복지관           전라북도 익산시 동천로7길 26-1   
1    관운사가야복지센터         경상북도 성주군 성주읍 경산길 33-1   
2  이웃과하나노인복지센터         경상북도 성주군 성주읍 예산3길 8-4   
3    용암종합사회복지관    충청북도 청주시 상당구 원봉로 52-1(용암동)   
4   청주시노인종합복지관  충청북도 청주시 상당구 대성로172번길 21(수동)   

                              소재지지번주소        운영기관명          전화번호         급식장소  \
0                                 NaN      동산사회복지관  063-842-2253      동산사회복지관   
1              경상북도 성주군 성주읍 경산리 577-4    관운사가야복지센터  054-931-3000    관운사가야복지센터   
2                경상북도 성주군 성주읍 예산리 466  이웃과하나노인복지센터  054-931-1611  이웃과하나노인복지센터   
3  충청북도 청주시 상당구 용암동 2108, 용암주공@ 1,2단지    용암종합사회복지관  043-293-9191    용암종합사회복지관   
4               충청북도 청주시 상당구 수동 138-8   청주시노인종합복지관  043-255-2144   청주시노인종합복지관   

                     급식대상             급식시간  급식요일      운영시작일자      운영종료일자  \
0               결식노인,저소득층  중식(12:00-13:00)     금  2017-01-01  2017-12-31   
1  60세이상 저소득층 독거노인 및 결식노인  중식(11:30~13:00)  월, 금  2010-01-08         NaN

In [8]:
# 운영종료된 급식소 표기 X
cafeteria['운영종료일자'].isnull()

0       False
1        True
2        True
3        True
4        True
        ...  
1387    False
1388    False
1389    False
1390    False
1391    False
Name: 운영종료일자, Length: 1392, dtype: bool

In [9]:
# 운영 종료일자 11/30 이상 종료일자가 없는 급식소 데이터 추출
cafeteria_on = cafeteria[(cafeteria['운영종료일자']> '2021-11-30') | (cafeteria['운영종료일자'].isnull() == True)]
cafeteria_on['운영종료일자'].value_counts()

2021-12-31    168
2022-12-31     79
2022-12-30     11
2050-12-31      5
2099-12-31      5
2021-12-30      4
2050-01-01      3
2025-12-31      2
Name: 운영종료일자, dtype: int64

In [10]:
# 위경도 결측치 제거
print(cafeteria_on['위도'].isnull().value_counts())
print(cafeteria_on['경도'].isnull().value_counts())

False    1087
True       40
Name: 위도, dtype: int64
False    1087
True       40
Name: 경도, dtype: int64


In [11]:
# 위도 결측치 제거
cafeteria_on = cafeteria_on[cafeteria_on['위도'].isnull() == False]

In [12]:
print(cafeteria_on['위도'].isnull().value_counts())
print(cafeteria_on['경도'].isnull().value_counts())

False    1087
Name: 위도, dtype: int64
False    1087
Name: 경도, dtype: int64


In [13]:
# 급식요일 값 통일하기 
cafeteria_on['급식요일'].value_counts()

월+화+수+목+금      429
월+화+수+목+금+토    184
월,화,수,목,금       41
월~금             31
월+수+금           20
              ... 
매주 수             1
2,3,4주 월         1
월+화+목+금          1
주1회(토)           1
월+화+수+목+금+일      1
Name: 급식요일, Length: 172, dtype: int64

In [14]:
# 통일하기
cafeteria_on['급식요일'] = cafeteria_on['급식요일'].str.replace('+', ',')
cafeteria_on['급식요일'] = cafeteria_on['급식요일'].str.replace('-', '~')
cafeteria_on['급식요일'].value_counts()

<ipython-input-14-149790b25646>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cafeteria_on['급식요일'] = cafeteria_on['급식요일'].str.replace('+', ',')


월,화,수,목,금      470
월,화,수,목,금,토    188
월~금             34
월~토             31
월,수,금           20
              ... 
2,3,4주 월         1
월,화,목,금          1
주1회(토)           1
주2회(월, 수)        1
월,화,수,목,금,일      1
Name: 급식요일, Length: 165, dtype: int64

In [15]:
#지도 시각화 필요한 라이브러리
import json
from pandas.io.json import json_normalize
import os
import folium
from folium import plugins
print(folium.__version__)

0.12.1


In [57]:
# 무료급식소 시각화 최종

from folium import plugins # 플러그인 라이브러리
import folium              # folium 지도 라이브러리
#import geocoder


from folium.plugins import MarkerCluster     # 군집 플러그인
from folium.plugins import MeasureControl    # 거리 측정 플러그인
from folium.plugins import Search            # 검색 플러그인
from folium.plugins import MousePosition     # 마우스포인트 -> 위경도 플러그인
from folium.plugins import Geocoder          # 검색 플러그인2

# 바탕 지도 만들기 
g_map = folium.Map(location = (cafeteria_on['위도'].mean(), cafeteria_on['경도'].mean()), tiles = 'OpenStreetMap',
                  zoom_start = 15, width = '100%', height = '100%')

# 마커 군집화 
marker_cluster = folium.plugins.MarkerCluster().add_to(g_map)


# 지도에 마커 표시하기
for name, call, time, day, p_type,lat, lng in zip(cafeteria_on.시설명,cafeteria_on.전화번호,cafeteria_on.급식시간,
                                                  cafeteria_on.급식요일,cafeteria_on.급식대상,
                                                  cafeteria_on.위도, cafeteria_on.경도):
    iframe = name+ '<br>'+'전화번호: '+ str(call) + '<br>' +'운영시간: '+ str(time) + '<br>' +'운영요일: '+ str(day) +  '<br>' +'급식대상: '+ str(p_type) 
    popup1 = folium.Popup(iframe, max_width = 650, max_height = 300)
    folium.Marker([lat, lng],icon = folium.Icon(color = 'purple',icon = 'cloud'), popup = popup1).add_to(marker_cluster)



# 검색 기능,,, 작동 안됨,,, 
Search(layer = marker_cluster,
       geom_type='Point',
       placeholder="장소 검색",
       search_label= 'name',
       search_zoom=30,
       collapsed=False,
       position='topright').add_to(g_map)  # 검색창 

# 기능 플러그인 
plugins.LocateControl(auto_start=True, strings={"title": "내 현재위치"}).add_to(g_map)    # 현위치 버튼
plugins.Geocoder(placeholder="장소 검색").add_to(g_map)                        # 검색 기능2
g_map.add_child(MeasureControl())                       # 거리 측정 버튼
MousePosition().add_to(g_map)                           # 마우스 포인트 기능


#folium.LayerControl(collapsed=False).add_to(g_map)     # 범주화 목록 

g_map.save('전국_무료_급식소_최종.html')  


#html 소스 출력
#html_gmap = g_map.get_root().render() 
#folium.Map().get_root().render() 를 이용하면 html 파일을 저장하는거 말고 html 소스를 리턴할 수 있다.
#out = open('전국_무료_급식소_소스.txt', 'w')
#print(html_gmap,file = out)

In [42]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Icon(color='blue', icon_color='white', icon='info-sign', angle=0, prefix='glyphicon', **kwargs)
 |  
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `

In [15]:
# 전국 아동 급식 지원 가맹점 
print(restaurant.head())
print(restaurant.info())

          가맹점명  가맹점유형코드    시도명 시군구명  시군구코드                      소재지도로명주소  \
0   GS25 엄궁동일점        3  부산광역시  사상구  33900       부산광역시 사상구 낙동대로 751(엄궁동)   
1  GS25 엄궁사거리점        3  부산광역시  사상구  33900       부산광역시 사상구 낙동대로 790(엄궁동)   
2   GS25 주례동서점        3  부산광역시  사상구  33900          부산광역시 사상구 주례로 7(주례동)   
3  GS25 주례무지개점        3  부산광역시  사상구  33900        부산광역시 사상구 주례로 101(주례동)   
4   GS25 주례여고점        3  부산광역시  사상구  33900  부산광역시 사상구 가야대로366번길119 (주례동)   

  소재지지번주소         위도          경도          전화번호  ... 토요일운영종료시각 공휴일운영시작시각  \
0     NaN  35.128450  128.969467  051-311-3383  ...     23:59     00:00   
1     NaN  35.131031  128.972449  051-323-3909  ...     23:59     00:00   
2     NaN  35.150397  129.011586  051-642-2277  ...     23:59     00:00   
3     NaN  35.146178  129.005201  051-928-0017  ...     23:59     00:00   
4     NaN  35.145766  129.013526  051-313-1758  ...     23:59     00:00   

  공휴일운영종료시각 배달시작시각 배달종료시각      관리기관명      관리기관전화번호     데이터기준일자   제공기관코드  \
0     23:59  00:0

In [46]:
# 아동 급식 지원 가맹점
# 결측치 정리
print(restaurant['위도'].isnull().value_counts())
print(restaurant['경도'].isnull().value_counts())

False    33215
True       639
Name: 위도, dtype: int64
False    33205
True       649
Name: 경도, dtype: int64


In [47]:
# 결측치 제거
restaurant_on = restaurant[restaurant['위도'].isnull() == False]
restaurant_on = restaurant_on[restaurant_on['경도'].isnull() == False]

In [48]:
print(restaurant_on['위도'].isnull().value_counts())
print(restaurant_on['경도'].isnull().value_counts())

False    33186
Name: 위도, dtype: int64
False    33186
Name: 경도, dtype: int64


In [49]:
# 운영시간 파생변수 생성 
cols = ['평일운영시작시각', '평일운영종료시각']
restaurant_on['평일운영시간'] = restaurant_on[cols].apply(lambda row: '~'. join(row.values.astype(str)), axis =1)
print(restaurant_on['평일운영시간'].head())

cols = ['토요일운영시작시각', '토요일운영종료시각']
restaurant_on['토요일운영시간'] = restaurant_on[cols].apply(lambda row: '~'. join(row.values.astype(str)), axis =1)
print(restaurant_on['토요일운영시간'].head())

cols = ['공휴일운영시작시각', '공휴일운영종료시각']
restaurant_on['공휴일운영시간'] = restaurant_on[cols].apply(lambda row: '~'. join(row.values.astype(str)), axis =1)
print(restaurant_on['공휴일운영시간'].head())

0    00:00~23:59
1    00:00~23:59
2    00:00~23:59
3    00:00~23:59
4    00:00~23:59
Name: 평일운영시간, dtype: object
0    00:00~23:59
1    00:00~23:59
2    00:00~23:59
3    00:00~23:59
4    00:00~23:59
Name: 토요일운영시간, dtype: object
0    00:00~23:59
1    00:00~23:59
2    00:00~23:59
3    00:00~23:59
4    00:00~23:59
Name: 공휴일운영시간, dtype: object


In [58]:
# 아동급식카드 가맹점 시각화 최종

import geocoder      
import folium


from folium.plugins import MarkerCluster    # 군집화 플러그인
from folium.plugins import MeasureControl   # 거리측정 플러그인
from folium.plugins import MousePosition     # 마우스포인트 -> 위경도 플러그인
from folium.plugins import Search           # 검색 플러그인
from folium.plugins import Geocoder         # 검색기능2 플러그인

# 바탕 지도 
g_map = folium.Map(location = (restaurant_on['위도'].mean(), restaurant_on['경도'].mean()), tiles = 'OpenStreetMap',
                  zoom_start = 15, width = '100%', height = '100%')

# 마커 군집화 
marker_cluster = MarkerCluster().add_to(g_map)


# 지도에 마커 생성
for name, call,time1,time2, lat, lng in zip(restaurant_on.가맹점명, restaurant_on.전화번호, restaurant_on.평일운영시간,
                                            restaurant_on.토요일운영시간, restaurant_on.위도, restaurant_on.경도):
    iframe = name +'<br>'+ '전화번호: '+ str(call)+ '<br>' + '평일운영시간: '+ str(time1) +'<br>' + '토요일운영시간: '+ str(time2) 
    popup1 = folium.Popup(iframe, max_width = 650, max_height = 300)
    folium.Marker([lat, lng],
                   icon = folium.Icon(icon = 'cloud',color = 'purple'),
                    popup = popup1
                  ).add_to(marker_cluster)

    
# 검색기능,,, 안됨,, 
Search(layer= marker_cluster,
       geom_type='Point',
       placeholder="장소 검색",
       search_label= 'name',
       search_zoom=30,
       collapsed=False,
       position='topright').add_to(g_map)  # 검색

plugins.LocateControl(auto_start=True, strings={"title": "내 현재위치"}).add_to(g_map)   # 현위치 버튼
plugins.Geocoder(placeholder="장소 검색").add_to(g_map)                       # 검색기능 2 
g_map.add_child(MeasureControl())                      # 거리측정 버튼
MousePosition().add_to(g_map)                           # 마우스 포인트 기능

g_map.save('전국_아동급식카드_가맹점_최종.html') 


#html 소스 출력
#html_gmap = g_map.get_root().render() 
#folium.Map().get_root().render() 를 이용하면 html 파일을 저장하는거 말고 html 소스를 리턴할 수 있다.
#out = open('전국_무료_급식소_소스.txt', 'w')
#print(html_gmap,file = out)

In [51]:
# 선한영항력 지도 
import geopandas as gpd
df = gpd.read_file('cafeteria/11102795_20211109205731.geojson', encoding = 'utf-8')
df.head()

C:\Users\Yuna park\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,id,CONTENTS_NAME,IMG_MAIN_URL,ADDR_OLD,ADDR_NEW,COORD_DATA,COORD_X,COORD_Y,TEL_NO,CONTENTS_DETAIL,...,LINE_COLOR,LINE_WEIGHT,MOVIE_URL,VOICE_URL,NATION_BASE_AREA,NATION_POINT_NUMBER,LAN_TYPE,KEYWORD,SUB_ID,geometry
0,0,종로1973,/smgis/ucimgs/conts/11102795/goodshop_0042_1_K...,강원도 춘천시 동내면 거두리 966-1,강원도 춘천시 춘천순환로72번길 33-14,,127.754851192,37.853638095,033-264-9286,,...,#0000FF,4,,,24405,라사22418379,KOR,,,GEOMETRYCOLLECTION (POINT (127.75490 37.85360))
1,1,골목닭갈비,/smgis/ucimgs/conts/11102795/goodshop_0042_1_K...,강원도 태백시 장성동 159-17,강원도 태백시 장성로 34-1,,129.009462946,37.101884761,033-581-7911,,...,#0000FF,4,,,26052,마사3413142,KOR,,,GEOMETRYCOLLECTION (POINT (129.00950 37.10190))
2,10,해장국마을(마이갓감자탕),/smgis/ucimgs/conts/11102795/goodshop_0042_1_K...,강원도 강릉시 강동면 안인진리 102-1,강원도 강릉시 안인진길 26,,128.988481654,37.734466536,033-645-0825,,...,#0000FF,4,,,25626,마사31157158,KOR,,,GEOMETRYCOLLECTION (POINT (128.98850 37.73450))
3,100,홍두깨샤브샤브만두전골,/smgis/ucimgs/conts/11102795/goodshop_0042_1_K...,경기도 군포시 산본동 1060-1,경기도 군포시 고산로 693,,126.935114143,37.371895668,031-399-2263,,...,#0000FF,4,,,15802,다사49983046,KOR,,,GEOMETRYCOLLECTION (POINT (126.93510 37.37190))
4,1000,금룡중화요리,/smgis/ucimgs/conts/11102795/goodshop_0042_1_K...,경상남도 창원시 성산구 반지동 78,경상남도 창원시 성산구 원이대로393번길 25,,128.662977907,35.239687673,055-276-5442,,...,#0000FF,4,,,51420,마라5819442,KOR,,,GEOMETRYCOLLECTION (POINT (128.66300 35.23970))


In [59]:
# 선한영항력 
from folium import plugins
import folium
import geocoder


from folium.plugins import MarkerCluster    # 군집화 플러그인
from folium.plugins import MeasureControl   # 거리측정 플러그인
from folium.plugins import MousePosition     # 마우스포인트 -> 위경도 플러그인
from folium.plugins import Search           # 검색 플러그인


# 바탕지도
g = geocoder.ip('me')
g_map = folium.Map(location= g.latlng, zoom_start = 15, width = '100%', height = '100%')
#g_map = folium.Map(location= (df['COORD_Y'].mean(),df['COORD_X'].mean()), zoom_start = 15, width = '100%', height = '100%')

marker_cluster = MarkerCluster().add_to(g_map)



# 지도 위 마커 생성
for name, call, lat, lng in zip(df.CONTENTS_NAME,df.TEL_NO,df.COORD_Y, df.COORD_X):
    iframe = name + '<br>' + '전화번호: '+ str(call)
    popup1 = folium.Popup(iframe, max_width = 650, max_height = 300)
    mark =folium.Marker([lat, lng],icon = folium.Icon(icon = 'cloud',color = 'purple'),popup = popup1).add_to(marker_cluster)



# 검색 연동을 위해 geojson 변환    
#marker_geojson = folium.GeoJson(df, overlay = False, control = False, show = False, embed = False).add_to(g_map)
marker_geojson = folium.GeoJson(df, marker = mark, popup = popup1 ,embed=False).add_to(g_map)


# 검색
Search(layer= marker_geojson,
       geom_type='Point',
       placeholder="장소 검색",
       search_label= 'CONTENTS_NAME',
       search_zoom=30,
       collapsed= False,
       position='topright').add_to(g_map)  


plugins.LocateControl(auto_start=True, strings={"title": "내 현재위치"}).add_to(g_map)   # 현위치 버튼
g_map.add_child(MeasureControl())                      # 거리측정 버튼
MousePosition().add_to(g_map)                           # 마우스 포인트 기능
#plugins.Geocoder().add_to(g_map)



g_map.save('선한영향력가게_최종.html') 
#html_gmap = g_map.get_root().render() 
#folium.Map().get_root().render() 를 이용하면 html 파일을 저장하는거 말고 html 소스를 리턴할 수 있다.

C:\Users\Yuna park\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Yuna park\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [40]:
#!pip install SpeechRecognition

In [43]:
#!pip install gtts

In [46]:
#!pip3 install playsound

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7026 sha256=54269edbe50b21b76dfe63ed24ec41acf37cd75a7b1ef8d07b281272dd4e7ea7
  Stored in directory: c:\users\yuna park\appdata\local\pip\cache\wheels\73\cd\cf\9750b618d54bd81c20e4c34fb24a423a5b095920367cdb3f71
Successfully built playsound


In [48]:
#!pip install pyttsx3  

In [41]:
import speech_recognition as sr
sr.__version__

'3.8.1'

In [55]:
import pyttsx3  
s = pyttsx3.init()  
data = "Sample Text"  
s.say(data)  
s.runAndWait()  

In [ ]:
for i in zip(cafeteria_on)